In [5]:
import json
import pandas as pd
import numpy as np
import cv2
import copy
from ultralytics import YOLO
import shutil
import os
import ast

In [114]:
with open('data/basketball-instants-dataset.json', 'r') as f:
    data = json.load(f)

In [115]:
basketball_data = []
player_data = []

for timestamp in data:
    ball_timestamp = copy.deepcopy(timestamp)
    player_timestamp = copy.deepcopy(timestamp)
    ball_timestamp['annotations'] = []
    player_timestamp['annotations'] = []
    basketball_data.append(ball_timestamp)
    player_data.append(player_timestamp)
    for annotation in timestamp['annotations']:
        if annotation['type'] == 'ball':
            ball_timestamp['annotations'].append(annotation)
        elif annotation['type'] == 'player':
            player_timestamp['annotations'].append(annotation)

In [116]:
with open('data/basketball-positions.json', "w") as f:
    json.dump(basketball_data, f)
with open('data/player-positions.json', "w") as f:
    json.dump(player_data, f)

In [6]:
with open('data/basketball-positions.json', 'r') as f:
    basketball_data = json.load(f)
with open('data/player-positions.json', 'r') as f:
    player_data = json.load(f)

In [7]:
count_with_ball = 0
count_with_player = 0
for timestamp in basketball_data:
    if timestamp['annotations'][0]['visible']:
        count_with_ball += 1
for timestamp in player_data:
    if len(timestamp['annotations']) > 0:
        count_with_player += 1
print("Number of images with ball visible: " + str(count_with_ball))
print("Number of images total: " + str(len(basketball_data)))
print("Number of images with players: " + str(count_with_player))

Number of images with ball visible: 320
Number of images total: 364
Number of images with players: 162


In [4]:
for frame in basketball_data:
    if frame['annotations'][0]['visible']:
        filename1 = 'camcourt1_' + str(frame['timestamp']) + '_' + '0.png'
        filename2 = 'camcourt1_' + str(frame['timestamp']) + '_' + '40.png'
        filename3 = 'camcourt2_' + str(frame['timestamp']) + '_' + '0.png'
        filename4 = 'camcourt2_' + str(frame['timestamp']) + '_' + '40.png'
        shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename1, os.path.join('yolo-basketball-data/images/', filename1))
        try:
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename2, os.path.join('yolo-basketball-data/images/', filename2))
        except FileNotFoundError:
            filename2 = filename2[:-6] + '80.png'
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename2, os.path.join('yolo-basketball-data/images/', filename2))
        shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename3, os.path.join('yolo-basketball-data/images/', filename3))
        try:
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename4, os.path.join('yolo-basketball-data/images/', filename4))
        except FileNotFoundError:
            filename4 = filename4[:-6] + '80.png'
            shutil.copy('data/' + str(frame['arena_label']) + '/' + str(frame['game_id']) + '/' + filename4, os.path.join('yolo-basketball-data/images/', filename4))



In [8]:
def str_to_dict(s):
    try:
        return ast.literal_eval(s)
    except ValueError:
        return None

ball_annotations = pd.read_csv('basketball_annotations.csv', converters={'region_shape_attributes': str_to_dict})
ball_annotations = ball_annotations[['filename', 'region_count', 'region_shape_attributes']]
ball_annotations = ball_annotations[ball_annotations['region_count'] == 1]
ball_annotations['is_training'] = np.concatenate([np.array([True] * int(0.8 * len(ball_annotations))), np.array([False] * (len(ball_annotations) - int(0.8 * len(ball_annotations))))])
ball_annotations['is_training'] = ball_annotations['is_training'].sample(frac=1).reset_index(drop=True)

In [9]:
ball_annotations.head()

,filename,region_count,region_shape_attributes,is_training
0,camcourt1_1512405572696_0.png,1,"{'name': 'rect', 'x': 1415, 'y': 1090, 'width'...",True
1,camcourt1_1512405572696_40.png,1,"{'name': 'rect', 'x': 1342, 'y': 1087, 'width'...",True
2,camcourt1_1512405932787_0.png,1,"{'name': 'rect', 'x': 535, 'y': 1092, 'width':...",True
3,camcourt1_1512405932787_40.png,1,"{'name': 'rect', 'x': 524, 'y': 1090, 'width':...",False
16,camcourt1_1512421952112_0.png,1,"{'name': 'rect', 'x': 1427, 'y': 921, 'width':...",False


In [47]:
for index, row in ball_annotations.iterrows():
    label_filepath = ""
    is_train = row['is_training']
    filename = row['filename']
    label_filename = filename[:-4] + ".txt"
    annotation = row['region_shape_attributes']
    height, width, _ = cv2.imread('yolo-basketball-data/images/' + filename).shape
    if is_train:
        label_filepath = os.path.join('yolo-basketball-data/train/labels', label_filename)
        shutil.move('yolo-basketball-data/images/' + filename, 'yolo-basketball-data/train/images')
    else:
        label_filepath = os.path.join('yolo-basketball-data/val/labels', label_filename)
        shutil.move('yolo-basketball-data/images/' + filename, 'yolo-basketball-data/val/images')
    normal_x = (annotation['x'] + annotation['width']/2) / width
    normal_y = (annotation['y'] + annotation['height']/2) / height
    normal_width = annotation['width'] / width
    normal_height = annotation['height'] / height
    with open(label_filepath, 'w') as f:
        f.write(f"0 {normal_x} {normal_y} {normal_width} {normal_height}")